In [1]:
import numpy as np
import cv2
import scipy
import matplotlib.pyplot as plt
import math
import os
from Utils.ImageUtils import *
from Utils.MiscUtils import *
from Utils.GeometryUtils import *
import random
  

In [11]:
def main():
    
    folder_name = "/home/sakshi/courses/ENPM673/sakshi_p2/Data/Project2_Dataset2/data_1/data"
    config_file_name = "/home/sakshi/courses/ENPM673/sakshi_p2/Data/Project2_Dataset2/data_1/camera_params.yaml"
    images = loadImages(folder_name)
    K, D = getCamera(config_file_name)

    old_right_line = None
    old_left_line = None
    while images:
        image = images.pop(0)
        
        h, w, _ = image.shape
        #undistort the image
        image_undistorted = cv2.undistort(image, K, D)
        image_overlay = image_undistorted.copy()

        #remove noise
        image_blur = cv2.GaussianBlur(image_undistorted, (5,5), 0)

        #get bin
        image_bin = extractWhite(image_blur, threshold = 250)

        #roi
        image_roi, cropped_image = getROI(image_bin, 0.45) 

        #extract points
        # filtered_image_roi = sobelXYThresh(image_roi)

        #get lines
        linesP = cv2.HoughLinesP(image_roi, 1, np.pi / 180, 50, None, 50, 10)

        #select left and right lines
        right_line, left_line = filterLines(linesP, old_right_line, old_left_line)
        old_right_line = right_line
        old_left_line = left_line

        # extend line
        extended_right_line = extendLines(right_line,  0.55 * h,  h)
        extended_left_line = extendLines(left_line,  0.55 * h,  h)

        # fill polygon
        corners = []
        
        corners.append([extended_left_line[0], extended_left_line[1]])
        corners.append([extended_right_line[0], extended_right_line[1]])
        corners.append([extended_right_line[2], extended_right_line[3]])
        corners.append([extended_left_line[2], extended_left_line[3]])
        corners = np.array(corners)

        cv2.fillPoly(image_overlay, pts =[corners], color=(0,0,255))

        #overlay
        cv2.addWeighted(image_overlay, 0.4, image, 0.6,
        0, image)
        

        #draw lines    
        cv2.line(image, (extended_right_line[0], extended_right_line[1]), (extended_right_line[2], extended_right_line[3]), (0, 0, 255), 3, cv2.LINE_AA)
        cv2.line(image, (extended_left_line[0], extended_left_line[1]), (extended_left_line[2], extended_left_line[3]), (0, 0, 255), 3, cv2.LINE_AA)

        cv2.imshow('frame', image)
        cv2.waitKey(10)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()

In [12]:
if __name__ == "__main__":
    main()


Loading images from  /home/sakshi/courses/ENPM673/sakshi_p2/Data/Project2_Dataset2/data_1/data


In [52]:
folder_name = "/home/sakshi/courses/ENPM673/sakshi_p2/Data/Project2_Dataset2/data_1/data"
config_file_name = "/home/sakshi/courses/ENPM673/sakshi_p2/Data/Project2_Dataset2/data_1/camera_params.yaml"
images = loadImages(folder_name)
K, D = getCamera(config_file_name)

Loading images from  /home/sakshi/courses/ENPM673/sakshi_p2/Data/Project2_Dataset2/data_1/data


In [53]:
image = images[1]
h, w, _ = image.shape
#undistort the image
image_undistorted = cv2.undistort(image, K, D)
image_overlay = image_undistorted.copy()

#remove noise
image_blur = cv2.GaussianBlur(image_undistorted, (5,5), 0)

#get bin
image_bin = extractWhite(image_blur, threshold = 250)

#roi
image_roi, cropped_image = getROI(image_bin, 0.45) 

#extract points
filtered_image_roi = sobelXYThresh(image_roi)

# #get lines
linesP = cv2.HoughLinesP(image_roi, 1, np.pi / 180, 50, None, 50, 20)
right_line, left_line = filterLines(linesP, None, None)

#draw lines    
cv2.line(image, (right_line[0], right_line[1]), (right_line[2], right_line[3]), (0, 0, 255), 3, cv2.LINE_AA)
cv2.line(image, (left_line[0], left_line[1]), (left_line[2], left_line[3]), (0, 0, 255), 3, cv2.LINE_AA)



cv2.imshow('frame', image)
cv2.waitKey()
cv2.destroyAllWindows()

NameError: name 'old_right_line' is not defined

In [50]:
cv2.imwrite('i_roi.jpg',image_roi)

True